In [61]:
!pip install requests boto3
!pip install python-dotenv


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [62]:
import requests
from urllib.parse import unquote
import boto3
import pandas as pd
from io import BytesIO, StringIO
import os 
from dotenv import load_dotenv

In [63]:
load_dotenv()
api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")
aws_access_key_id=os.getenv('S3_AWS_ACCESS_KEY_ID')
aws_secret_access_key=os.getenv('S3_AWS_SECRET_ACCESS_KEY') + '+6pPnSu'
region_name=os.getenv('S3_REGION')
bucket_name = os.getenv('S3_BUCKET_NAME')

# API 엔드포인트와 저장할 파일명 딕셔너리
api_endpoints = {
    "/15071311/v1/uddi:a5158b81-27c7-4151-ba6c-b912a6f13d39": "prod_data/서울교통공사_혼잡도_20171231.csv",
    "/15071311/v1/uddi:70e3a3d3-0872-4828-8234-f0bca459b44f": "prod_data/서울교통공사_지하철혼잡도정보_20191231.csv",
    "/15071311/v1/uddi:b3803d43-ffe3-4d17-9024-fd6cfa37c284": "prod_data/서울교통공사_지하철혼잡도정보_20211231.csv",
    "/15071311/v1/uddi:75461a18-17a3-42fe-9322-a51148003b69": "prod_data/서울교통공사_지하철혼잡도정보_20221231.csv",
    "/15071311/v1/uddi:99771417-a036-46f1-8ad5-8edf4591c2ee": "prod_data/서울교통공사_지하철혼잡도정보_20201231.csv",
    "/15071311/v1/uddi:e477f1d9-2c3a-4dc8-b147-a55584583fa2": "prod_data/서울교통공사_지하철혼잡도정보_20231231.csv",
    "/15071311/v1/uddi:c87b6af0-0ef7-4182-b172-fd2680a79d6f": "prod_data/서울교통공사_지하철혼잡도정보_20240331.csv",
    "/15071311/v1/uddi:9aff0ee6-26e7-42c4-af0c-84bf31680ca9": "prod_data/서울교통공사_지하철혼잡도정보_20240630.csv",
    "/15071311/v1/uddi:da7cd08f-94f0-4dba-b33d-d02dcb35b57b": "prod_data/서울교통공사_지하철혼잡도정보_20240930.csv"
}

In [65]:
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

bucket_name = bucket_name

In [66]:
def fetch_and_save_data(api_endpoint, file_name):
    all_data = pd.DataFrame()
    page = 1
    per_page = 1000

    while True:
        url = f"{base_url}{api_endpoint}?page={page}&perPage={per_page}&serviceKey={api_key}"

        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if 'data' in data and data['data']:
                df = pd.DataFrame(data['data'])
                all_data = pd.concat([all_data, df], ignore_index=True)
                print(f"{file_name} - Page {page} data fetched and added.")
            else:
                print(f"No more data to fetch for {file_name}.")
                break
        else:
            print(f"Failed to fetch data from {file_name} on page {page}: {response.status_code}")
            break

        page += 1

    csv_buffer = BytesIO()
    all_data.to_csv(csv_buffer, index=False, encoding='utf-8')
    csv_buffer.seek(0)

    try:
        s3_client.upload_fileobj(csv_buffer, bucket_name, file_name)
        print(f"{file_name} 파일이 S3에 성공적으로 업로드되었습니다.")
    except Exception as e:
        print(f"Failed to upload {file_name} to S3: {e}")

for endpoint, file_name in api_endpoints.items():
    fetch_and_save_data(endpoint, file_name)

prod_data/서울교통공사_혼잡도_20171231.csv - Page 1 data fetched and added.
prod_data/서울교통공사_혼잡도_20171231.csv - Page 2 data fetched and added.
No more data to fetch for prod_data/서울교통공사_혼잡도_20171231.csv.
prod_data/서울교통공사_혼잡도_20171231.csv 파일이 S3에 성공적으로 업로드되었습니다.
prod_data/서울교통공사_지하철혼잡도정보_20191231.csv - Page 1 data fetched and added.
prod_data/서울교통공사_지하철혼잡도정보_20191231.csv - Page 2 data fetched and added.
No more data to fetch for prod_data/서울교통공사_지하철혼잡도정보_20191231.csv.
prod_data/서울교통공사_지하철혼잡도정보_20191231.csv 파일이 S3에 성공적으로 업로드되었습니다.
prod_data/서울교통공사_지하철혼잡도정보_20211231.csv - Page 1 data fetched and added.
prod_data/서울교통공사_지하철혼잡도정보_20211231.csv - Page 2 data fetched and added.
No more data to fetch for prod_data/서울교통공사_지하철혼잡도정보_20211231.csv.
prod_data/서울교통공사_지하철혼잡도정보_20211231.csv 파일이 S3에 성공적으로 업로드되었습니다.
prod_data/서울교통공사_지하철혼잡도정보_20221231.csv - Page 1 data fetched and added.
prod_data/서울교통공사_지하철혼잡도정보_20221231.csv - Page 2 data fetched and added.
No more data to fetch for prod_data/서울교통공사_지하철혼잡도정보_20221